In [18]:
import torch
import sqlite3
import pandas as pd

In [19]:
conn = sqlite3.connect('db.db')
data = pd.read_sql_query("SELECT * from main", conn)

### how can we predict multiple swell heights and directions? 

- each time stamp (every 30 minutes) has the dominant swell height. We want to predict this
- for a 1 day prediction. get all the data as late as one day ago, and this is a data point with many dimensions. Maybe grab a snapshot of each buoy from 
- encode this into a large vector. Use WvHT (waveheight), MWD (main swell direction in degrees), ans SwP as prediction. 
- put this in a neural network because this will be non linear and high dimensional

### parsing data and building model